<a href="https://colab.research.google.com/github/IvanHG-OffSec/SemilleroIA/blob/main/Ejercicio_Final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

HERRERA GOMEZ IGNACIO IVAN
EJRECICIO FINAL

In [13]:
# LIBRERIAS
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import Pipeline

In [8]:


# CARGA DE DATPS
url = "https://archive.ics.uci.edu/ml/machine-learning-databases/statlog/german/german.data"
data = pd.read_csv(url, sep=" ", header=None)

# COLUMNAS A ESPAÑOL
data.columns = [
    'cuenta_corriente', 'duracion_credito_meses', 'historial_credito', 'proposito',
    'monto_credito', 'cuenta_ahorro', 'antiguedad_empleo', 'tasa_cuota',
    'estado_civil_sexo', 'garantes', 'tiempo_residencia', 'propiedades',
    'edad', 'otros_creditos', 'vivienda', 'creditos_existentes',
    'profesion', 'personas_a_cargo', 'telefono', 'extranjero', 'confiable'
]

# VARIABLES DE OBJETIVO BINARIO
data['confiable'] = data['confiable'].map({1: 1, 2: 0})

# VARIABLES DE ENTRADA Y SALIDA
X = data.drop(columns='confiable')
Y = data['confiable']

# PARTICION 85% ENTRENAMIENTO Y 15% PRUEBAS
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.15, random_state=1, stratify=Y)

# MOSTRAR LOS RESULTADOS
print(X_train.shape, ': dimensión de datos de entrada de entrenamiento')
print(X_test.shape, ': dimensión de datos de entrada de prueba')
print(y_train.shape, ': dimensión de variable de salida de entrenamiento')
print(y_test.shape, ': dimensión de variable de salida de prueba')


(850, 20) : dimensión de datos de entrada de entrenamiento
(150, 20) : dimensión de datos de entrada de prueba
(850,) : dimensión de variable de salida de entrenamiento
(150,) : dimensión de variable de salida de prueba


In [9]:

vars_numericas = ['duracion_credito_meses', 'monto_credito', 'edad']
vars_ordinales = ['tasa_cuota', 'tiempo_residencia', 'creditos_existentes', 'personas_a_cargo']
vars_categoricas = ['cuenta_corriente', 'historial_credito', 'proposito', 'cuenta_ahorro','antiguedad_empleo', 'estado_civil_sexo', 'garantes', 'propiedades',
'profesion', 'vivienda']
vars_binarias = ['telefono', 'extranjero']

imputador_numerico = Pipeline(steps=[('imputacion', SimpleImputer(strategy='mean'))])

imputador_ordinal = Pipeline(steps=[('imputacion', SimpleImputer(strategy='median'))])

imputador_categorico = Pipeline(steps=[('imputacion', SimpleImputer(strategy='most_frequent'))])

imputador_binario = Pipeline(steps=[('imputacion', SimpleImputer(strategy='most_frequent'))])

preprocesamiento = ColumnTransformer(transformers=[
    ('num', imputador_numerico, vars_numericas),
    ('ord', imputador_ordinal, vars_ordinales),
    ('cat', imputador_categorico, vars_categoricas),
    ('bin', imputador_binario, vars_binarias)
])

# PARA LOS DATOS DE ENTRENAMIENTO
X_train_imputado = preprocesamiento.fit_transform(X_train)

print("Forma de los datos imputados:", X_train_imputado.shape)


Forma de los datos imputados: (850, 19)


In [11]:

# VARIABLES NUMERICAS
imputador_y_escalador_numerico = Pipeline(steps=[
    ('imputacion', SimpleImputer(strategy='mean')),
    ('escalado', MinMaxScaler())
])

preprocesamiento = ColumnTransformer(transformers=[
    ('num', imputador_y_escalador_numerico, vars_numericas),
    ('ord', imputador_ordinal, vars_ordinales),
    ('cat', imputador_categorico, vars_categoricas),
    ('bin', imputador_binario, vars_binarias)
])

# DATOS PROS.
X_train_preproc = preprocesamiento.fit_transform(X_train)
print("Forma de datos tras imputación y escalado:", X_train_preproc.shape)


Forma de datos tras imputación y escalado: (850, 19)


In [12]:
from sklearn.preprocessing import OneHotEncoder

# TRANSF. ONE-HOT
pipeline_cat_bin = Pipeline(steps=[
    ('imputacion', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

pipeline_num = Pipeline(steps=[
    ('imputacion', SimpleImputer(strategy='mean')),
    ('escalado', MinMaxScaler())
])

pipeline_ord = Pipeline(steps=[
    ('imputacion', SimpleImputer(strategy='median'))
])

# TRASN. FINA;
preprocesamiento_final = ColumnTransformer(transformers=[
    ('num', pipeline_num, vars_numericas),
    ('ord', pipeline_ord, vars_ordinales),
    ('cat_bin', pipeline_cat_bin, vars_categoricas + vars_binarias)
])

# SE APICA AL ENTRENAMIENTO
X_train_preproc = preprocesamiento_final.fit_transform(X_train)

print("Dimensiones de X_train después de preprocesamiento:", X_train_preproc.shape)


Dimensiones de X_train después de preprocesamiento: (850, 58)
